# Přísnost známkování na TIMSS datech

Trends in International Mathematics and Science Study: https://en.wikipedia.org/wiki/Trends_in_International_Mathematics_and_Science_Study

Cílem je prozkoumat rozdíly v přísnosti známkování mezi školami na základě jejich různých charakteristik -- například kraj, velikost sídla apod. TIMSS data obsahují testové skóry a známky pro matematiku a přírodovědu (science), navíc lze dataset napojit s doplňujícími informacemi o škole.

Testové skóry jsou v datasetu s pěti plausible values - žáci zpravidla vyplňují pouze část z celé testové baterie, plausible values jsou nutné pro správné odhady směrodatných chyb. Metodologie týkající se plausible values v TIMSS a dalších mezinárodních vzdělávacích studiích je někde určitě popsaná důkladněji, v analýze níže řeším pouze bodové odhady založené na průměru všech plausible values (což by mělo být metodologicky korektní).

Odhady pro školy jsou založené na fixed effects modelu, který je v ekonometrii poměrně častý. Pokud si dobře vzpomínám, je v zásadě ekvivalentní s odlišnými intercepts pro jednotlivé školy v obyčejném lineárním modelu. Výhledově sem zkusím doplnit více matematiky. Nejsem si stoprocentně jistý, zdali je tento postup ideální pro analýzu rozdílů v přísnosti známkování, ale zatím jsem nevymyslel nic lepšího.

Předběžné závěry naznačují, že rozdíly ve známkování jsou spíše menší.

## Analýza

In [ ]:
# nejake standardni importy

import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.ioff()

In [ ]:
# toto spravne importuje doplnujici knihovnu - muj kod, ktery pouzivam pro vytvareni html stranek z nekolika grafu
# mozna je ale jednodussi pouzivat proste normalni grafy v notebooku
cwd = os.getcwd()
utils_dir = os.path.join(cwd, '..', 'python-utils')
sys.path.append(utils_dir)

from libs.utils import *
from libs.plots import *
from libs.extensions import *

# cesty je nutne si upravit, podle toho kde se skutecne nachazi temp dir a data!
os.environ['TEMP'] = '/mnt/d/temp'

In [ ]:
# cesta k datum
data_root = '/mnt/d/projects/idea/data'

In [ ]:
# nacteni dat - skoro vse k mezinarodnim vzdelavacim studiim je ve formatu .sav pro SPSS
# zaroven pro SPSS existuje i plugin pro korektni praci s plausible values, nicmene pro nase ucely
# se to tolik nehodi a navic nemam legalni SPSS...
sg11, sg11_meta = pyreadstat.read_sav(f'{data_root}/TIMSS/2011/CZ/T11_dot_zak.sav')
sg15, sg15_meta = pyreadstat.read_sav(f'{data_root}/TIMSS/2015/CZ/ASGCZEM6.sav')

In [ ]:
sg11

In [ ]:
shared_cols = {
    'IDSCHOOL': 'school',
    'IDCLASS': 'class',
    'IDSTUD': 'student',
    'ITSEX': 'girl',
    'TOTWGT': 'weight',
    **{f'ASMMAT0{i}': f'mat_pv{i}' for i in range(1, 6)},
    **{f'ASSSCI0{i}': f'sci_pv{i}' for i in range(1, 6)}
}

cols11 = {**shared_cols, 'o02c': 'mat_grade', 'o02d': 'sci_grade' }
cols15 = {**shared_cols, 'ASXG20C': 'mat_grade', 'ASXG20D': 'sci_grade' }

sg11 = sg11[cols11.keys()].rename(columns=cols11)
sg15 = sg15[cols15.keys()].rename(columns=cols15)

for c in ['school', 'class', 'student', 'girl']:
    sg11[c] = pd.Series(sg11[c], dtype=np.int_)
    sg15[c] = pd.Series(sg15[c], dtype=np.int_)

sg11['girl'] = -sg11['girl'] + 2
sg15['girl'] = -sg15['girl'] + 2

for c in ['mat', 'sci']:
    sg11[c + '_grade'] = np.where(sg11[c + '_grade'] > 5, np.nan, sg11[c + '_grade'])
    sg15[c + '_grade'] = np.where(sg15[c + '_grade'] > 5, np.nan, sg15[c + '_grade'])
    sg11[c] = np.mean(sg11[[f'{c}_pv{i}' for i in range(1, 6)]], axis=1)
    sg15[c] = np.mean(sg15[[f'{c}_pv{i}' for i in range(1, 6)]], axis=1)

sg11['unit'] = 1
sg15['unit'] = 1

In [ ]:
sg11

In [ ]:
cg15, cg15_meta = pyreadstat.read_sav(f'{data_root}/TIMSS/2015/CZ/ACGCZEM6.sav')
cg11, cg11_meta = pyreadstat.read_sav(f'{data_root}/TIMSS/2011/CZ/T11_dot_skolni.sav')

In [ ]:
# okomentuji nekdy pozdeji :)

def get_weighted_means(frame, weight_col='weight', cols=None):
    res = pd.Series(dtype='float64')
    for c in cols or frame.columns[frame.columns != weight_col]:
        res[c] = (frame[c] * frame[weight_col]).sum() / frame[weight_col].sum()
    return res

def get_fixed_effects(df, group_col, grade_col, score_col, girl_col='girl', weight_col='weight'):
    df = df[[group_col, grade_col, score_col, girl_col, weight_col]].dropna().copy()
    stats_grade = DescrStatsW(df[grade_col], weights=df[weight_col])
    stats_score = DescrStatsW(df[score_col], weights=df[weight_col])

    df[score_col] = (df[score_col] - stats_score.mean) / stats_score.std
    df[grade_col] = -(df[grade_col] - stats_grade.mean) / stats_grade.std

    cols = [grade_col, score_col, girl_col]
    c_means = df.groupby(group_col).apply(get_weighted_means, weight_col=weight_col, cols=cols).reset_index()
    df = pd.merge(df, c_means.rename(columns={c: c + '_cm' for c in cols}))
    for c in cols:
        df[c + '_adj'] = df[c] - df[c + '_cm'] + np.average(df[c], weights=df[weight_col])

    model = sm.WLS(df[grade_col + '_adj'], sm.add_constant(df[[score_col + '_adj', girl_col + '_adj']]), weights=df[weight_col]).fit()

    c_means['fe'] = c_means[grade_col] - model.params['const'] \
        - model.params[score_col + '_adj'] * c_means[score_col] \
        - model.params[girl_col + '_adj'] * c_means[girl_col]
    c_means['count'] = df.groupby(group_col)[grade_col].count().values

    return c_means[[group_col, 'fe', 'count']]

In [ ]:
fe15_mat = get_fixed_effects(sg15, 'school', 'mat_grade', 'mat')
fe15_sci = get_fixed_effects(sg15, 'school', 'sci_grade', 'sci')
fe11_mat = get_fixed_effects(sg11, 'school', 'mat_grade', 'mat')
fe11_sci = get_fixed_effects(sg11, 'school', 'sci_grade', 'sci')

fe15_mat_class = get_fixed_effects(sg15, 'class', 'mat_grade', 'mat')
fe15_sci_class = get_fixed_effects(sg15, 'class', 'sci_grade', 'sci')
fe11_mat_class = get_fixed_effects(sg11, 'class', 'mat_grade', 'mat')
fe11_sci_class = get_fixed_effects(sg11, 'class', 'sci_grade', 'sci')

foo = sg15[['mat_grade', 'weight']].dropna()
stats_grade = DescrStatsW(foo['mat_grade'], weights=foo['weight'])
stats_grade.std

cg15['school'] = np.int_(cg15['IDSCHOOL'])
cg11['school'] = np.int_(cg11['IDSCHOOL'])

# what do I need for these plots? ---

def compare_fe(cg, cg_meta, var, col, label, num_labels, left=0.16, title=None, fe_col='fe'):
    labels = [cg_meta.value_labels[label][x] for x in range(1, num_labels + 1)]
    cg[var] = pd.Categorical(cg[col].apply(lambda x: None if not np.isfinite(x)
        else cg_meta.value_labels[label][x]), labels, ordered=True)
    plt.rcParams['figure.figsize'] = 10, 6
    plt.rcParams['figure.subplot.left'] = left
    fig, ax = plt.subplots()
    sns.boxplot(y=var, x=fe_col, data=cg)
    if title is not None:
        ax.set_title(title)
    return ax

vars = ['population', 'area', 'poor', 'affluent']
cols = ['ACBG05A', 'ACBG05B', 'ACBG03A', 'ACBG03B']
labels = ['labels{}'.format(i) for i in [5, 6, 2, 3]]
num_labels15 = [7, 5, 4, 4]
num_labels11 = [6, 5, 4, 4]
lefts15 = [0.16, 0.33, 0.16, 0.16]
lefts11 = [0.24, 0.16, 0.16, 0.16]

def compare_fe_all(cg, cg_meta, vars, cols, labels, num_labels, lefts, title):
    plots = []
    for v, c, l, nl, left in zip(vars, cols, labels, num_labels, lefts):
        ax = compare_fe(cg, cg_meta, v, c, l, nl, left=left)
        ax.set_title(title.format(v))
        plots.append(ax)
    return plots


plots15_mat = compare_fe_all(pd.merge(cg15, fe15_mat), cg15_meta, vars, cols, labels, num_labels15, lefts15,
    title='TIMSS 2015, Maths, schools FE vs {}')
plots15_sci = compare_fe_all(pd.merge(cg15, fe15_sci), cg15_meta, vars, cols, labels, num_labels15, lefts15,
    title='TIMSS 2015, Science, schools FE vs {}')
plots11_mat = compare_fe_all(pd.merge(cg11, fe11_mat), cg11_meta, vars, cols, labels, num_labels11, lefts11,
    title='TIMSS 2011, Maths, schools FE vs {}')
plots11_sci = compare_fe_all(pd.merge(cg11, fe11_sci), cg11_meta, vars, cols, labels, num_labels11, lefts11,
    title='TIMSS 2011, Science, schools FE vs {}')

_, size11_mat = plt.subplots()
sns.scatterplot(x='ACBG01', y='fe', data=pd.merge(cg11, fe11_mat))
size11_mat.set(xlabel='size of school', title='TIMSS 2011, Maths, schools FE vs size of school')
plots11_mat.append(size11_mat)

_, size11_sci = plt.subplots()
sns.scatterplot(x='ACBG01', y='fe', data=pd.merge(cg11, fe11_sci))
size11_sci.set(xlabel='size of school', title='TIMSS 2011, Science, schools FE vs size of school')
plots11_sci.append(size11_sci)


In [ ]:
# toto ve WSL ted nefunguje; pri jinem spusteni mozna ano, ale nemam vyzkousene v notebooku
# navic je stejne potreba opravit cross-origin policy ve firefoxu, aby to fungovalo...

Selector([
    Chart(plots15_mat, title='Maths, 2015', cols=2),
    Chart(plots15_sci, title='Science, 2015', cols=2),
    Chart(plots11_mat, title='Maths, 2011', cols=2),
    Chart(plots11_sci, title='Science, 2011', cols=2)
], title='TIMSS, schools FE vs schools characteristics').show()

In [ ]:
def hist_fe(fe, weights=False, title=''):
    stat = DescrStatsW(fe['fe'], weights=(fe['count'] if weights else None))
    fig, ax = plt.subplots()
    ax.hist(fe['fe'], bins=20, rwidth=0.9, weights=(fe['count'] if weights else None))
    ax.set_title('{} (std dev = {:.3g})'.format(title, stat.std))
    return ax

plt.rcParams['figure.figsize'] = 10, 6

fe_hists = []
for y in [15, 11]:
    for c in ['mat', 'sci']:
        for g in ['school', 'class']:
            for w in [False, True]:
                fe = eval('fe{}_{}{}'.format(y, c, ('' if g == 'school' else '_class')))
                title = 'TIMSS, 20{}, {}, {} FE{}'.format(y, ('Maths' if c == 'mat' else 'Science'), g,
                    (', by #students' if w else ''))
                ax = hist_fe(fe, w, title)
                fe_hists.append(ax)

Chart(fe_hists, title='TIMSS, histograms of FE', cols=2).show()